In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./1.category.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['1st Category No'] = input_['1st Category No'].astype(int)
input_['2nd Category No'] = input_['2nd Category No'].astype(int)
input_['3rd Category No'] = input_['3rd Category No'].astype(int)

print(f'总数量：{len(input_)}')
print()

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)

# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error

    while not work.empty():
        a = work.get_nowait()

        # = = = = = = = = = = = = = = =

        crawler_status = 'error'

        # = = = = = = = = = = = = = = =

        import requests

        import sys
        sys.path.append('../00.Tools')
        from crawler_configuration import get_header, get_proxy

        from bs4 import BeautifulSoup
        from lxml import etree

        try:
            df_temp, page = pd.DataFrame(), 1
            while True:
                page_url, b = f'''{input_.loc[a, '3rd Category Url']}?sort=alphaasc&limit=96&page={page}''', 0
                while True:
                    b += 1
                    
                    try:
                        resp = requests.get(page_url,
                                            headers=get_header(),
                                            proxies=get_proxy(),
                                            timeout=(10, 10))

                        if resp.status_code == 200:
                            soup = BeautifulSoup(resp.text, 'lxml')
                            html = etree.HTML(str(soup))

                            # = = = = = = = = = = = = = = =

                            if input_.loc[a, '3rd Category'] in html.xpath('//title/text()')[0]:
                                break
                    except:
                        continue

                # = = = = = = = = = = = = = = =

                list_url = html.xpath('//h4[@class="card-title NoMargin"]/a/@href')

                # = = = = = = = = = = = = = = =

                df_temp_temp = pd.DataFrame({'1st Category No': input_.loc[a, '1st Category No'],
                                             '1st Category': input_.loc[a, '1st Category'],
                                             '1st Category Url': input_.loc[a, '1st Category Url'],
                                             '2nd Category No': input_.loc[a, '2nd Category No'],
                                             '2nd Category': input_.loc[a, '2nd Category'],
                                             '2nd Category Url': input_.loc[a, '2nd Category Url'],
                                             '3rd Category No': input_.loc[a, '3rd Category No'],
                                             '3rd Category': input_.loc[a, '3rd Category'],
                                             '3rd Category Url': input_.loc[a, '3rd Category Url'],
                                             'Page': page,
                                             'Page Url': page_url,
                                             'No': [page*96-95+i for i in range(len(list_url))],
                                             'Url': list_url})

                df_temp = pd.concat([df_temp, df_temp_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =

                if work.qsize() % 100 == 0 and work.qsize() != 0:
                    clear_output()

                print(f'''[尝试次数：{b}] - {input_.loc[a, '1st Category No']}.{input_.loc[a, '1st Category']} > {input_.loc[a, '2nd Category No']}.{input_.loc[a, '2nd Category']} > {input_.loc[a, '3rd Category No']}.{input_.loc[a, '3rd Category']} > Page {page}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

                # = = = = = = = = = = = = = = =

                page += 1
                
                # = = = = = = = = = = = = = = =

                if not html.xpath('//li[@class="pagination-item pagination-item--next"]'):
                    df_temp['No'] = [i+1 for i in range(len(df_temp))]

                    break
                    
            # = = = = = = = = = = = = = = =

            if df_temp.empty:
                raise

            # = = = = = = = = = = = = = = =

            output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')
            
            # = = = = = = = = = = = = = = =

            crawler_status = 'ok'
        except:
            df_temp = pd.DataFrame([{'1st Category No': input_.loc[a, '1st Category No'],
                                     '1st Category': input_.loc[a, '1st Category'],
                                     '1st Category Url': input_.loc[a, '1st Category Url'],
                                     '2nd Category No': input_.loc[a, '2nd Category No'],
                                     '2nd Category': input_.loc[a, '2nd Category'],
                                     '2nd Category Url': input_.loc[a, '2nd Category Url'],
                                     '3rd Category No': input_.loc[a, '3rd Category No'],
                                     '3rd Category': input_.loc[a, '3rd Category'],
                                     '3rd Category Url': input_.loc[a, '3rd Category Url']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['1st Category No', '2nd Category No', '3rd Category No', 'Page', 'No'],
                                                ascending=[True, True, True, True, True],
                                                ignore_index=True)
    output_correct.to_excel(f'''./2.url-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
if not output_error.empty:
    output_error = output_error.sort_values(by=['1st Category No', '2nd Category No', '3rd Category No'],
                                            ascending=[True, True, True],
                                            ignore_index=True)
    output_error.to_excel('./url_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：112

[尝试次数：2] - 1.Brakes > 2.Brake Boosters & Pumps > 1.Electronic Brake Assembly > Page 1
[剩余数量：92] - [当前时间：19:51:31]

[尝试次数：17] - 1.Brakes > 1.ABS Components > 3.ABS Pump & Motor Assembly > Page 1
[剩余数量：91] - [当前时间：19:51:31]

[尝试次数：7] - 1.Brakes > 1.ABS Components > 1.ABS Control Module > Page 1
[剩余数量：90] - [当前时间：19:51:32]

[尝试次数：15] - 1.Brakes > 3.Disc Brakes > 5.Parking Brake Actuator > Page 1
[剩余数量：90] - [当前时间：19:51:32]

[尝试次数：10] - 2.Drivetrain > 1.Differential > 1.4WD Actuator > Page 1
[剩余数量：89] - [当前时间：19:51:34]

[尝试次数：6] - 1.Brakes > 2.Brake Boosters & Pumps > 7.Vacuum Pump > Page 1
[剩余数量：88] - [当前时间：19:51:34]

[尝试次数：31] - 1.Brakes > 3.Disc Brakes > 3.Brake Caliper Bracket > Page 1
[剩余数量：87] - [当前时间：19:51:34]

[尝试次数：9] - 1.Brakes > 2.Brake Boosters & Pumps > 6.Power Brake Booster Hydraulic Motor Pump > Page 1
[剩余数量：87] - [当前时间：19:51:35]

[尝试次数：32] - 1.Brakes > 3.Disc Brakes > 4.Brake Dust Shield > Page 1
[剩余数量：86] - [当前时间：19:51:35]

[尝试次数：28] - 2.Drivetrain > 1.Differentia

KeyError: 'Part No'